In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import json
import os
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

MEMORY_FILE = 'smart_agent_memory.json'

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
if os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, 'r') as f:
        memory = json.load(f)
else:
    memory = {}

In [ ]:
def save_memory():
    with open(MEMORY_FILE, 'w') as f:
        json.dump(memory, f, indent=4)

In [ ]:
def get_most_similar(user_history, question, threshold=0.7):
    if not user_history:
        return None

    questions = list(user_history.keys())
    embeddings = model.encode(questions + [question])
    similarities = util.cos_sim(embeddings[-1], embeddings[:-1])[0]

    best_idx = similarities.argmax().item()
    best_score = similarities[best_idx].item()

    if best_score > threshold:
        return questions[best_idx]
    return None

In [ ]:
def get_response(name, question):
    user_data = memory.get(name, {"history": {}})
    history = user_data["history"]

    similar_question = get_most_similar(history, question)

    if similar_question:
        return history[similar_question]
    else:
        print("I don't know the answer. Can you teach me?")
        answer = input("Your answer: ")
        history[question] = answer
        memory[name] = {"history": history}
        save_memory()
        return answer


In [ ]:
print("=== Smart AI Agent ===")
user = input("Your name: ")

while True:
    q = input(f"{user}, ask something (or type 'exit'): ")
    if q.lower() == "exit":
        print("Bye!")
        break

    ans = get_response(user, q)
    print("AI:", ans)

=== Smart AI Agent ===
Your name: somnath
somnath, ask something (or type 'exit'): What is the color of sky ?
I don't know the answer. Can you teach me?
Your answer: the color is blue
AI: the color is blue
somnath, ask something (or type 'exit'): what is the sky's color ?
AI: the color is blue
somnath, ask something (or type 'exit'): exit
Bye!
